# More on Functional Programming

## Using higher-order functions


Collecting results of functions

In [1]:
; A function that returns the square of each element of a list

(defn square [x]
    (* x x))

(defn square-all [numbers]
    (if (empty? numbers)
        nil 
    (cons (square (first numbers))
          (square-all (rest numbers)))))

(square-all [1 2 3 4 5 6])


(1 4 9 16 25 36)

In [2]:
; A function that returns the cube of each element of a list

(defn cube [x]
    (* x x x))

(defn cube-all [numbers]
    (if (empty? numbers)
        ()
    (cons (cube (first numbers))
          (cube-all (rest numbers)))))

(cube-all [1 2 3 4 5 6])

(1 8 27 64 125 216)

In [3]:
; A HOF 'do-to-all' that generalizes tha las 2 functions

(defn do-to-all [func numbers]
    (if (empty? numbers)
        ()
    (cons (func (first numbers))
          (do-to-all func (rest numbers)))))


#'user/do-to-all

In [4]:
; The new function 'do-to-all' can get the square of each element

(do-to-all square [1 2 3 4 5 6])

(1 4 9 16 25 36)

In [5]:
; The new function 'do-to-all' can get the cube of each element

(do-to-all cube [1 2 3 4 5 6])


(1 8 27 64 125 216)

In [6]:
; In summary, 'do-to-all' is a custom implementation of 'map'

(map cube [1 2 3 4 5 6])


(1 8 27 64 125 216)

In [7]:
; But 'do-to-all' blows the stack with large numbers

(do-to-all square (range 11000))


Execution error (StackOverflowError) at user/do-to-all (REPL:6).
null


class java.lang.StackOverflowError: 

In [17]:
; A new implementation of 'do-to-all' using lazy sequences

(defn do-to-all-2 [func numbers]
    (lazy-seq 
        (if (empty? numbers)
            ()
        (cons (func (first numbers))
              (do-to-all-2 func (rest numbers))))))


#'user/do-to-all-2

In [18]:
; Testing with a large number again (doesn't blow the stack anymore)

(take 10 (drop 10000 (do-to-all-2 square (range 11000))))


(100000000 100020001 100040004 100060009 100080016 100100025 100120036 100140049 100160064 100180081)

Reduce list of things

In [19]:
; A function that returns the sum of a list of numbers

(defn total-of [numbers]
    (loop [nums numbers sum 0]
        (if (empty? nums)
            sum
            (recur (rest nums) (+ sum (first nums))))))

(total-of [5 7 9 3 4 1 2 8])

39

In [20]:
; A couple of functions that returns the biggest number of a list of numbers

(defn larger-of [x y]
    (if (> x y) x y))

(defn largest-of [numbers]
    (loop [l numbers candidate (first numbers)]
        (if (empty? l)
            candidate
            (recur (rest l) (larger-of candidate (first l))))))

(largest-of [5 7 9 3 4 1 2 8])

9

In [21]:
; A HOF 'compute-across' that generalizes the last 2 functions

(defn compute-across [func elements value]
    (if (empty? elements)
        value
        (recur func (rest elements) (func value (first elements)))))


#'user/compute-across

In [22]:
; 'compute-across' can compute the sum of a list of numbers

(defn total-of-2 [numbers]
    (compute-across + numbers 0))

(total-of-2 [5 7 9 3 4 1 2 8])

39

In [23]:
; 'compute-across' can also get the biggest number of a list of numbers

(defn largest-of-2 [numbers]
    (compute-across larger-of numbers (first numbers)))

(largest-of [5 7 9 3 4 1 2 8])


9

In [24]:
; ; In summary, 'compute-across' is a custom implementation of 'reduce'

(reduce + [5 7 9 3 4 1 2 8])

39

In [25]:
(reduce max [5 7 9 3 4 1 2 8])


9

Filtering lists of things


In [26]:
; A function that returns a list of the numbers greater than a given number

(defn all-greater-than [threshold numbers]
    (compute-across #(if (> %2 threshold) (conj %1 %2) %1) numbers [])) ; '%' are the anon function args

(all-greater-than 5 [5 7 9 3 4 1 2 8])

[7 9 8]

In [27]:
; A function that returns a list of the numbers smaller than a given number

(defn all-lesser-than [threshold numbers]
    (compute-across #(if (< %2 threshold) (conj %1 %2) %1) numbers []))

(all-lesser-than 5 [5 7 9 3 4 1 2 8])

[3 4 1 2]

In [28]:
; A HOF 'select-if' that generalizes the 2 last functions

(defn select-if [pred elements]
    (compute-across #(if (pred %2) (conj %1 %2) %1) elements []))

#'user/select-if

In [29]:
; 'select-if' can get all the even numbers

(select-if even? [5 7 9 3 4 1 2 8])

[4 2 8]

In [30]:
; 'select-if' can get all the odd numbers

(select-if odd? [5 7 9 3 4 1 2 8])

[5 7 9 3 1]

In [31]:
; 'all-greater-than' can be implemented in terms of 'select'if'

(defn all-greater-than-v2 [threshold numbers]
    (select-if #(> % threshold) numbers))

(all-greater-than-v2 5 [5 7 9 3 4 1 2 8])

[7 9 8]

In [32]:
; 'all-lesser-than' can be implemented in terms of 'select'if'

(defn all-lesser-than-v2 [threshold numbers]
    (select-if #(< % threshold) numbers))


(all-lesser-than-v2 5 [5 7 9 3 4 1 2 8])

[3 4 1 2]

In [33]:
; In summary, 'select-if' is a custom implementation of 'filter'

(filter even? [5 7 9 3 4 1 2 8])

(4 2 8)

In [34]:
(filter odd? [5 7 9 3 4 1 2 8])

(5 7 9 3 1)

## Partial application


In [35]:
; A tax calculator function

(defn price-with-tax [tax-rate amount]
    (->> (/ tax-rate 100)
         (+ 1)
         (* amount)))

(price-with-tax 9.5M 100)


109.500M

In [36]:
; The last function can be used to get the prices
; accounting for California's and New York's tax rates
-
(defn price-with-ca-tax [price]
    (price-with-tax 9.25M price))

(defn price-with-ny-tax [price]
    (price-with-tax 8.0M price))

(def prices [100 200 300 400 500])


#'user/prices

In [37]:
(map price-with-ca-tax prices)


(109.2500M 218.5000M 327.7500M 437.0000M 546.2500M)

In [38]:
(map price-with-ny-tax prices)


(108.00M 216.00M 324.00M 432.00M 540.00M)

In [39]:
; A function 'price-calculator-for-tax' that generalizes 'price-with-ca-tax' and 'price-with-ny-tax'

(defn price-calculator-for-tax [state-tax]
    (fn [price]
        (price-with-tax state-tax price)))




#'user/price-calculator-for-tax

In [40]:
; 'price-with-ca-tax-2' is a 'partially applied' function 
; with CA's tax rate as a partial arg

(def price-with-ca-tax-2 (price-calculator-for-tax 9.25M)) ; Note the function definition as a var

; The results are the same as the 1st version

(map price-with-ca-tax-2 prices)


(109.2500M 218.5000M 327.7500M 437.0000M 546.2500M)

In [41]:
; 'price-with-ny-tax-2' is a 'partially applied' function 
; with NY's tax rate as a partial arg

(def price-with-ny-tax-2 (price-calculator-for-tax 8.0M)) ; Note the function definition as a var
; The results are the same as the 1st version

(map price-with-ny-tax-2 prices)


(108.00M 216.00M 324.00M 432.00M 540.00M)

In [42]:
; A simple function of arity 5 that concatenates the args as a string

(defn of-n-args [a b c d e] 
    (str a b c d e ))


#'user/of-n-args

In [43]:
; Partially applying to the last function the args a, b, c

(defn of-k-args [d e]
    (of-n-args 1 2 3 d e))

(of-k-args \a \b)


"123ab"

In [44]:
; A general form of a function that can partially apply 
; 'n-minus-k-args' to the function 'of-n-args' 

(defn partially-applied [of-n-args & n-minus-k-args]
    (fn [& k-args]
        (apply of-n-args (concat n-minus-k-args k-args))))


#'user/partially-applied

In [58]:
; Partially applying the first 3 args of 'of-n-args'

(def of-2-args (partially-applied of-n-args \a \b \c))

(of-2-args 4 5)


"abc45"

In [59]:
; Partially applying the first 2 args of 'of-n-args'

(def of-3-args (partially-applied of-n-args \a \b))

(of-3-args 3 4 5)


"ab345"

In [60]:
; The function 'of-n-args' is a custom implementation of the 'partial' Clojure function

(def of-2-args (partial of-n-args \a \b \c))

(of-2-args 4 5)

"abc45"

In [48]:
(def of-3-args (partial of-n-args \a \b))

(of-3-args 3 4 5)


"ab345"

In [61]:
; The select-if function defined in the previous section

(defn select-if [pred elements]
    (compute-across #(if (pred %2) (conj %1 %2) %1) elements []))


#'user/select-if

In [62]:
; Rewriting 'select-if' to take an arbitrary initial container

(defn select-into-if [container pred elements]
    (compute-across #(if (pred %2) (conj %1 %2) %1) elements container))

(def numbers [4 9 5 7 6 3 8])


#'user/numbers

In [52]:
; Passing a vector as initial value to 'select-into-if' and evaluating it

(select-into-if [] #(< % 7) numbers)


[4 5 6 3]

In [63]:
; Passing a list as initial value to 'select-into-if' and evaluating it
; (see how the order of the sequence changes)

(select-into-if () #(< % 7) numbers)


(3 6 5 4)

In [55]:
; The 'select-up' function can be defined to return the filtered results in the same order,
; using a vector as the initial container

(def select-up (partial select-into-if []))

(select-up #(< % 9) [5 3 9 6 8])



[5 3 6 8]

In [64]:
; The 'select-down' function can be defined to return the filtered results with the inverted order,
; using a list as the initial container

(def select-down (partial select-into-if ()))

(select-down #(< % 9) [5 3 9 6 8])


(8 6 3 5)